In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
stress_raw = pd.read_csv('stress.csv')
stress_raw.head()

,Support,Run,stressor,Bogo Ops,Real Time (s),Usr Time (s),Sys Time (s),Bogo Ops/s (Real Time),Bogo Ops/s (Usr+Sys Time)
0,vm,1,cpu,55992,60.00,59.94,0.03,933.13,933.67
1,vm,1,cpu,55819,60.01,59.82,0.08,930.14,931.87
2,vm,1,cpu,55792,60.00,59.84,0.08,929.84,931.11
3,vm,1,cpu,55777,60.00,59.90,0.03,929.57,930.7
4,vm,2,cpu,54447,60.00,59.72,0.22,907.39,908.36


In [11]:
stress_raw = stress_raw[stress_raw['Usr Time (s)'] != 0]

In [12]:
# Remove commas and convert to numeric for relevant columns in stress_raw
cols_with_commas = ['Bogo Ops', 'Bogo Ops/s (Real Time)', 'Bogo Ops/s (Usr+Sys Time)']
for col in cols_with_commas:
    stress_raw[col] = stress_raw[col].replace({',': ''}, regex=True)
    stress_raw[col] = pd.to_numeric(stress_raw[col], errors='coerce')

In [15]:
# Filter for stressor == 'vm'
df_vm = stress_raw[stress_raw['stressor'] == 'vm']

# Columns to aggregate
cols = [
    'Bogo Ops',
    'Real Time (s)',
    'Usr Time (s)',
    'Sys Time (s)',
    'Bogo Ops/s (Real Time)',
    'Bogo Ops/s (Usr+Sys Time)'
]

# Convert columns to numeric (if not already)
for col in cols:
    df_vm[col] = pd.to_numeric(df_vm[col], errors='coerce')

# Group by 'Support' and aggregate mean and std
agg_df = df_vm.groupby('Support')[cols].agg(['mean', 'std']).reset_index()
agg_df

/tmp/ipykernel_16224/1724675150.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vm[col] = pd.to_numeric(df_vm[col], errors='coerce')


Support      Bogo Ops                Real Time (s)            \
                      mean            std          mean       std   
0  container  3.163024e+06  308413.527425     60.616667  0.938528   
1       host  3.275806e+06  285000.954546     60.808235  1.028064   
2         vm  2.417147e+06    9431.509700     60.155500  0.385042   

  Usr Time (s)           Sys Time (s)           Bogo Ops/s (Real Time)  \
          mean       std         mean       std                   mean   
0    37.762778  4.609236     9.936667  1.291010           52180.350000   
1    41.280588  4.107109    11.102941  1.213393           53899.871765   
2    48.854000  2.631118     9.516000  1.328309           40182.686500   

               Bogo Ops/s (Usr+Sys Time)               
           std                      mean          std  
0  5024.639649              66543.321667  2954.414005  
1  4947.845998              62686.966471  3254.219624  
2   265.417465              41433.749500   951.507057

In [17]:
agg_df_transposed = agg_df.set_index(('Support', '')).transpose()
agg_df_transposed

('Support', '')                    container          host            vm
Bogo Ops                  mean  3.163024e+06  3.275806e+06  2.417147e+06
                          std   3.084135e+05  2.850010e+05  9.431510e+03
Real Time (s)             mean  6.061667e+01  6.080824e+01  6.015550e+01
                          std   9.385283e-01  1.028064e+00  3.850424e-01
Usr Time (s)              mean  3.776278e+01  4.128059e+01  4.885400e+01
                          std   4.609236e+00  4.107109e+00  2.631118e+00
Sys Time (s)              mean  9.936667e+00  1.110294e+01  9.516000e+00
                          std   1.291010e+00  1.213393e+00  1.328309e+00
Bogo Ops/s (Real Time)    mean  5.218035e+04  5.389987e+04  4.018269e+04
                          std   5.024640e+03  4.947846e+03  2.654175e+02
Bogo Ops/s (Usr+Sys Time) mean  6.654332e+04  6.268697e+04  4.143375e+04
                          std   2.954414e+03  3.254220e+03  9.515071e+02

In [21]:
def agg_df_to_latex_pm(df):
    # Get support types (columns)
    supports = df.columns.tolist()
    # Get metrics (row MultiIndex)
    metrics = df.index.get_level_values(0).unique()
    # Build header
    header = "Metric & " + " & ".join(supports) + " \\\\\n\\hline\n"
    lines = [header]
    for metric in metrics:
        mean_row = df.loc[(metric, 'mean')]
        std_row = df.loc[(metric, 'std')]
        line = [metric]
        for support in supports:
            mean = mean_row[support]
            std = std_row[support]
            cell = f"${mean:.2f} \\pm {std:.2f}$"
            line.append(cell)
        lines.append(" & ".join(line) + " \\\\\n")
    latex_table = "\\begin{tabular}{l" + "c" * len(supports) + "}\n"
    latex_table += "".join(lines)
    latex_table += "\\end{tabular}"
    print(latex_table)

agg_df_to_latex_pm(agg_df_transposed)

\begin{tabular}{lccc}
Metric & container & host & vm \\
\hline
Bogo Ops & $3163023.94 \pm 308413.53$ & $3275805.65 \pm 285000.95$ & $2417147.20 \pm 9431.51$ \\
Real Time (s) & $60.62 \pm 0.94$ & $60.81 \pm 1.03$ & $60.16 \pm 0.39$ \\
Usr Time (s) & $37.76 \pm 4.61$ & $41.28 \pm 4.11$ & $48.85 \pm 2.63$ \\
Sys Time (s) & $9.94 \pm 1.29$ & $11.10 \pm 1.21$ & $9.52 \pm 1.33$ \\
Bogo Ops/s (Real Time) & $52180.35 \pm 5024.64$ & $53899.87 \pm 4947.85$ & $40182.69 \pm 265.42$ \\
Bogo Ops/s (Usr+Sys Time) & $66543.32 \pm 2954.41$ & $62686.97 \pm 3254.22$ & $41433.75 \pm 951.51$ \\
\end{tabular}
